In [ ]:
import requests
import isodate
import os
import re
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# API 키 가져오기
load_dotenv()
API_KEY = os.getenv("API_KEY")

# 검색 기간: 현재로부터 한 달 전 ~ 현재
current_date_obj = datetime.now(timezone.utc)
one_month_ago = current_date_obj - timedelta(days=30)
start_date = one_month_ago.isoformat()
end_date = current_date_obj.isoformat()

# ISO8601 duration 변환
def convert_duration_to_seconds(duration):
    try:
        parsed_duration = isodate.parse_duration(duration)
        seconds = int(parsed_duration.total_seconds()) if parsed_duration else 0
        return seconds
    except Exception as e:
        print(f"Duration 변환 오류: {e}")
        return 0

# 한국어 포함 여부 확인 함수
def contains_korean(text):
    return bool(re.search(r"[가-힣]", text))

# 카테고리 설정
category_id = 1

# 검색 API를 통해 모든 페이지의 영상 ID를 수집 (검색 조건에 맞는 영상만, 한국어 제목 포함)
def search_video_ids(category_id):
    search_url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "part": "snippet",
        "type": "video",
        "regionCode": "KR",
        "videoCategoryId": category_id,
        "order": "viewCount",
        "maxResults": 50,  # 최대 허용값
        "publishedAfter": start_date,
        "publishedBefore": end_date,
        "relevanceLanguage": "ko",
        "key": API_KEY,
    }
    
    video_ids = set()
    next_page_token = None
    while True:
        if next_page_token:
            params["pageToken"] = next_page_token
        response = requests.get(search_url, params=params)
        if response.status_code != 200:
            print(f"[카테고리 {category_id}] API 호출 실패: {response.text}")
            break
        data = response.json()
        for item in data.get("items", []):
            vid = item.get("id", {}).get("videoId")
            title = item.get("snippet", {}).get("title", "")
            if vid and contains_korean(title):
                video_ids.add(vid)
        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break
    return list(video_ids)

# 영상 세부 정보를 가져오기
def get_videos_details(video_ids):
    url = "https://www.googleapis.com/youtube/v3/videos"
    all_videos = []
    
    for i in range(0, len(video_ids), 50):
        batch = video_ids[i:i+50]
        params = {
            "part": "snippet,statistics,contentDetails",
            "id": ",".join(batch),
            "key": API_KEY,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"API 호출 오류: {response.text}")
            continue
        data = response.json()
        all_videos.extend(data.get("items", []))
    
    return all_videos

# 채널 정보를 가져오기
def get_channels_info(channel_ids):
    url = "https://www.googleapis.com/youtube/v3/channels"
    all_channels = []
    channel_ids = list(channel_ids)
    
    for i in range(0, len(channel_ids), 50):
        batch = channel_ids[i:i+50]
        params = {
            "part": "snippet,statistics",
            "id": ",".join(batch),
            "key": API_KEY,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"get_channels_info 호출 오류: {response.text}")
            continue
        data = response.json()
        all_channels.extend(data.get("items", []))
    
    return all_channels

# --- 메인 데이터 수집 ---
video_data = []
channel_ids = set()

print(f"카테고리 ID {category_id}에서 영상 수집 중...")
video_ids = search_video_ids(category_id)
print(f"검색된 영상 수: {len(video_ids)}")

if not video_ids:
    print(f"[카테고리 {category_id}] 조건에 맞는 영상이 없습니다.")
else:
    videos_details = get_videos_details(video_ids)
    print(f"상세 정보 가져온 영상 수: {len(videos_details)}")
    
    # 각 영상별로 조건에 맞는 데이터만 수집
    for video in videos_details:
        snippet = video.get("snippet", {})
        statistics = video.get("statistics", {})
        content_details = video.get("contentDetails", {})

        video_id = video.get("id")
        channel_id = snippet.get("channelId", "")
        title = snippet.get("title", "")
        view_count = int(statistics.get("viewCount", "0"))
        like_count = statistics.get("likeCount", "0")
        comment_count = statistics.get("commentCount", "0")
        upload_date = snippet.get("publishedAt", "")
        duration_sec = convert_duration_to_seconds(content_details.get("duration", "PT0S"))
        thumbnail_url = snippet.get("thumbnails", {}).get("high", {}).get("url", "")
        tags = snippet.get("tags", [])

        if duration_sec == 0:
            continue

        video_data.append([
            video_id, category_id, channel_id, title, view_count, like_count, comment_count, 
            upload_date, duration_sec, tags, thumbnail_url
        ])
        
        channel_ids.add(channel_id)

# 채널 정보 수집
channel_data = []
channels_details = get_channels_info(channel_ids)
for channel in channels_details:
    snippet = channel.get("snippet", {})
    statistics = channel.get("statistics", {})
    
    channel_data.append([
        channel.get("id"), snippet.get("title", ""), statistics.get("subscriberCount", "0"),
        statistics.get("viewCount", "0"), statistics.get("videoCount", "0")
    ])

# 데이터 저장 (CSV 파일로 저장)
df_videos = pd.DataFrame(video_data, columns=["videoID", "categoryID", "channelID", "title", "viewCount", "likeCount", "commentCount", "uploadDate", "duration", "tags", "thumbnailURL"])
df_videos.to_csv("videos_1.csv", index=False, encoding="utf-8-sig")
print("영상 데이터 저장 완료: videos_1.csv")

df_channels = pd.DataFrame(channel_data, columns=["channelID", "channelTitle", "subscriberCount", "totalViews", "videosCount"])
df_channels.to_csv("channel_1.csv", index=False, encoding="utf-8-sig")
print("채널 데이터 저장 완료: channel_1.csv")

print("완료!")

: 

In [ ]:
import requests
import isodate
import os
import re
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# API 키 가져오기
load_dotenv()
API_KEY = os.getenv("API_KEY")

# 검색 기간: 현재로부터 한 달 전 ~ 현재
current_date_obj = datetime.now(timezone.utc)
one_month_ago = current_date_obj - timedelta(days=30)
start_date = one_month_ago.isoformat()
end_date = current_date_obj.isoformat()

# ISO8601 duration 변환 및 4분 이상 영상만 허용 (240초)
def convert_duration_to_seconds(duration):
    try:
        parsed_duration = isodate.parse_duration(duration)
        seconds = int(parsed_duration.total_seconds()) if parsed_duration else 0
        return seconds if seconds >= 240 else 0
    except Exception as e:
        print(f"Duration 변환 오류: {e}")
        return 0

# 한국어 포함 여부 확인 함수
def contains_korean(text):
    return bool(re.search(r"[가-힣]", text))

# 카테고리 설정('Autos & Vehicles')
category_id = 2

# 검색 API를 통해 모든 페이지의 영상 ID를 수집 (검색 조건에 맞는 영상만, 한국어 제목 포함)
def search_video_ids(category_id):
    search_url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "part": "snippet",
        "type": "video",
        "regionCode": "KR",
        "videoCategoryId": category_id,
        "order": "viewCount",
        "maxResults": 50,  # 최대 허용값
        "publishedAfter": start_date,
        "publishedBefore": end_date,
        "relevanceLanguage": "ko",
        "key": API_KEY,
    }
    
    video_ids = set()
    next_page_token = None
    while True:
        if next_page_token:
            params["pageToken"] = next_page_token
        response = requests.get(search_url, params=params)
        if response.status_code != 200:
            print(f"[카테고리 {category_id}] API 호출 실패: {response.text}")
            break
        data = response.json()
        for item in data.get("items", []):
            vid = item.get("id", {}).get("videoId")
            title = item.get("snippet", {}).get("title", "")
            if vid and contains_korean(title):
                video_ids.add(vid)
        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break
    return list(video_ids)

# 영상 세부 정보를 가져오기
def get_videos_details(video_ids):
    url = "https://www.googleapis.com/youtube/v3/videos"
    all_videos = []
    
    for i in range(0, len(video_ids), 50):
        batch = video_ids[i:i+50]
        params = {
            "part": "snippet,statistics,contentDetails",
            "id": ",".join(batch),
            "key": API_KEY,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"API 호출 오류: {response.text}")
            continue
        data = response.json()
        all_videos.extend(data.get("items", []))
    
    return all_videos

# 채널 정보를 가져오기
def get_channels_info(channel_ids):
    url = "https://www.googleapis.com/youtube/v3/channels"
    all_channels = []
    channel_ids = list(channel_ids)
    
    for i in range(0, len(channel_ids), 50):
        batch = channel_ids[i:i+50]
        params = {
            "part": "snippet,statistics",
            "id": ",".join(batch),
            "key": API_KEY,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"get_channels_info 호출 오류: {response.text}")
            continue
        data = response.json()
        all_channels.extend(data.get("items", []))
    
    return all_channels

# --- 메인 데이터 수집 ---
video_data = []
channel_ids = set()

print(f"카테고리 ID {category_id}에서 영상 수집 중...")
video_ids = search_video_ids(category_id)
print(f"검색된 영상 수: {len(video_ids)}")

if not video_ids:
    print(f"[카테고리 {category_id}] 조건에 맞는 영상이 없습니다.")
else:
    videos_details = get_videos_details(video_ids)
    print(f"상세 정보 가져온 영상 수: {len(videos_details)}")
    
    # 각 영상별로 조건에 맞는 데이터만 수집
    for video in videos_details:
        snippet = video.get("snippet", {})
        statistics = video.get("statistics", {})
        content_details = video.get("contentDetails", {})

        video_id = video.get("id")
        channel_id = snippet.get("channelId", "")
        title = snippet.get("title", "")
        view_count = int(statistics.get("viewCount", "0"))
        like_count = statistics.get("likeCount", "0")
        comment_count = statistics.get("commentCount", "0")
        upload_date = snippet.get("publishedAt", "")
        duration_sec = convert_duration_to_seconds(content_details.get("duration", "PT0S"))
        thumbnail_url = snippet.get("thumbnails", {}).get("high", {}).get("url", "")
        tags = snippet.get("tags", [])

        if duration_sec == 0:
            continue

        video_data.append([
            video_id, category_id, channel_id, title, view_count, like_count, comment_count, 
            upload_date, duration_sec, tags, thumbnail_url
        ])
        
        channel_ids.add(channel_id)

# 채널 정보 수집
channel_data = []
channels_details = get_channels_info(channel_ids)
for channel in channels_details:
    snippet = channel.get("snippet", {})
    statistics = channel.get("statistics", {})
    
    channel_data.append([
        channel.get("id"), snippet.get("title", ""), statistics.get("subscriberCount", "0"),
        statistics.get("viewCount", "0"), statistics.get("videoCount", "0")
    ])

# 데이터 저장 (CSV 파일로 저장)
df_videos = pd.DataFrame(video_data, columns=["videoID", "categoryID", "channelID", "title", "viewCount", "likeCount", "commentCount", "uploadDate", "duration", "tags", "thumbnailURL"])
df_videos.to_csv("videos_2.csv", index=False, encoding="utf-8-sig")
print("영상 데이터 저장 완료: videos_2.csv")

df_channels = pd.DataFrame(channel_data, columns=["channelID", "channelTitle", "subscriberCount", "totalViews", "videosCount"])
df_channels.to_csv("channel_2.csv", index=False, encoding="utf-8-sig")
print("채널 데이터 저장 완료: channel_2.csv")

print("완료!")

In [ ]:
import requests
import isodate
import os
import re
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# API 키 가져오기
load_dotenv()
API_KEY = os.getenv("API_KEY")

# 검색 기간: 현재로부터 한 달 전 ~ 현재
current_date_obj = datetime.now(timezone.utc)
one_month_ago = current_date_obj - timedelta(days=30)
start_date = one_month_ago.isoformat()
end_date = current_date_obj.isoformat()

# ISO8601 duration 변환
def convert_duration_to_seconds(duration):
    try:
        parsed_duration = isodate.parse_duration(duration)
        seconds = int(parsed_duration.total_seconds()) if parsed_duration else 0
        return seconds
    except Exception as e:
        print(f"Duration 변환 오류: {e}")
        return 0

# 한국어 포함 여부 확인 함수
def contains_korean(text):
    return bool(re.search(r"[가-힣]", text))

# 카테고리 설정
category_id = 22

# 검색 API를 통해 모든 페이지의 영상 ID를 수집 (검색 조건에 맞는 영상만, 한국어 제목 포함)
def search_video_ids(category_id):
    search_url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "part": "snippet",
        "type": "video",
        "regionCode": "KR",
        "videoCategoryId": category_id,
        "order": "viewCount",
        "maxResults": 50,  # 최대 허용값
        "publishedAfter": start_date,
        "publishedBefore": end_date,
        "relevanceLanguage": "ko",
        "key": API_KEY,
    }
    
    video_ids = set()
    next_page_token = None
    while True:
        if next_page_token:
            params["pageToken"] = next_page_token
        response = requests.get(search_url, params=params)
        if response.status_code != 200:
            print(f"[카테고리 {category_id}] API 호출 실패: {response.text}")
            break
        data = response.json()
        for item in data.get("items", []):
            vid = item.get("id", {}).get("videoId")
            title = item.get("snippet", {}).get("title", "")
            if vid and contains_korean(title):
                video_ids.add(vid)
        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break
    return list(video_ids)

# 영상 세부 정보를 가져오기
def get_videos_details(video_ids):
    url = "https://www.googleapis.com/youtube/v3/videos"
    all_videos = []
    
    for i in range(0, len(video_ids), 50):
        batch = video_ids[i:i+50]
        params = {
            "part": "snippet,statistics,contentDetails",
            "id": ",".join(batch),
            "key": API_KEY,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"API 호출 오류: {response.text}")
            continue
        data = response.json()
        all_videos.extend(data.get("items", []))
    
    return all_videos

# 채널 정보를 가져오기
def get_channels_info(channel_ids):
    url = "https://www.googleapis.com/youtube/v3/channels"
    all_channels = []
    channel_ids = list(channel_ids)
    
    for i in range(0, len(channel_ids), 50):
        batch = channel_ids[i:i+50]
        params = {
            "part": "snippet,statistics",
            "id": ",".join(batch),
            "key": API_KEY,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"get_channels_info 호출 오류: {response.text}")
            continue
        data = response.json()
        all_channels.extend(data.get("items", []))
    
    return all_channels

# --- 메인 데이터 수집 ---
video_data = []
channel_ids = set()

print(f"카테고리 ID {category_id}에서 영상 수집 중...")
video_ids = search_video_ids(category_id)
print(f"검색된 영상 수: {len(video_ids)}")

if not video_ids:
    print(f"[카테고리 {category_id}] 조건에 맞는 영상이 없습니다.")
else:
    videos_details = get_videos_details(video_ids)
    print(f"상세 정보 가져온 영상 수: {len(videos_details)}")
    
    # 각 영상별로 조건에 맞는 데이터만 수집
    for video in videos_details:
        snippet = video.get("snippet", {})
        statistics = video.get("statistics", {})
        content_details = video.get("contentDetails", {})

        video_id = video.get("id")
        channel_id = snippet.get("channelId", "")
        title = snippet.get("title", "")
        view_count = int(statistics.get("viewCount", "0"))
        like_count = statistics.get("likeCount", "0")
        comment_count = statistics.get("commentCount", "0")
        upload_date = snippet.get("publishedAt", "")
        duration_sec = convert_duration_to_seconds(content_details.get("duration", "PT0S"))
        thumbnail_url = snippet.get("thumbnails", {}).get("high", {}).get("url", "")
        tags = snippet.get("tags", [])

        if duration_sec == 0:
            continue

        video_data.append([
            video_id, category_id, channel_id, title, view_count, like_count, comment_count, 
            upload_date, duration_sec, tags, thumbnail_url
        ])
        
        channel_ids.add(channel_id)

# 채널 정보 수집
channel_data = []
channels_details = get_channels_info(channel_ids)
for channel in channels_details:
    snippet = channel.get("snippet", {})
    statistics = channel.get("statistics", {})
    
    channel_data.append([
        channel.get("id"), snippet.get("title", ""), statistics.get("subscriberCount", "0"),
        statistics.get("viewCount", "0"), statistics.get("videoCount", "0")
    ])

# 데이터 저장 (CSV 파일로 저장)
df_videos = pd.DataFrame(video_data, columns=["videoID", "categoryID", "channelID", "title", "viewCount", "likeCount", "commentCount", "uploadDate", "duration", "tags", "thumbnailURL"])
df_videos.to_csv("videos_2.csv", index=False, encoding="utf-8-sig")
print("영상 데이터 저장 완료: videos_22.csv")

df_channels = pd.DataFrame(channel_data, columns=["channelID", "channelTitle", "subscriberCount", "totalViews", "videosCount"])
df_channels.to_csv("channel_22.csv", index=False, encoding="utf-8-sig")
print("채널 데이터 저장 완료: channel_22.csv")

print("완료!")

카테고리 ID 22에서 영상 수집 중...
검색된 영상 수: 475
상세 정보 가져온 영상 수: 475
영상 데이터 저장 완료: videos_22.csv
채널 데이터 저장 완료: channel_2.csv
완료!
